<a href="https://colab.research.google.com/github/DevilNReality/RedBus-Hackathon/blob/main/Code%20File%20/%20RedBus_Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and Dataset

In [ ]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
! unzip '/content/train_JDXlpm8.zip'

Archive:  /content/train_JDXlpm8.zip
   creating: train/
  inflating: train/train.csv         
  inflating: train/transactions.csv  


In [ ]:
train = pd.read_csv('/content/train/train.csv')
trans = pd.read_csv('/content/train/transactions.csv')

# Data Auditing

In [ ]:
train.head()

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0
3,2023-03-01,47,45,2580.0
4,2023-03-01,46,9,4185.0


In [ ]:
trans.head().T

,0,1,2,3,4
doj,2023-03-01,2023-03-01,2023-03-01,2023-03-01,2023-03-01
doi,2023-01-30,2023-01-30,2023-01-30,2023-01-30,2023-01-30
srcid,45,46,45,47,46
destid,46,45,47,45,9
srcid_region,Karnataka,Tamil Nadu,Karnataka,Andhra Pradesh,Tamil Nadu
destid_region,Tamil Nadu,Karnataka,Andhra Pradesh,Karnataka,Tamil Nadu
srcid_tier,Tier 1,Tier 1,Tier 1,Tier 1,Tier 1
destid_tier,Tier 1,Tier 1,Tier 1,Tier 1,Tier2
cumsum_seatcount,8.0,8.0,4.0,0.0,9.0
cumsum_searchcount,76.0,70.0,142.0,68.0,162.0


In [ ]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2266100 entries, 0 to 2266099
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   doj                 object 
 1   doi                 object 
 2   srcid               int64  
 3   destid              int64  
 4   srcid_region        object 
 5   destid_region       object 
 6   srcid_tier          object 
 7   destid_tier         object 
 8   cumsum_seatcount    float64
 9   cumsum_searchcount  float64
 10  dbd                 int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 190.2+ MB


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doj              67200 non-null  object 
 1   srcid            67200 non-null  int64  
 2   destid           67200 non-null  int64  
 3   final_seatcount  67200 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.1+ MB


# Data Filtering - Transaction

In [ ]:
trans1 = trans[trans['dbd'] > 14]

In [ ]:
trans1.head()

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30
3,2023-03-01,2023-01-30,47,45,Andhra Pradesh,Karnataka,Tier 1,Tier 1,0.0,68.0,30
4,2023-03-01,2023-01-30,46,9,Tamil Nadu,Tamil Nadu,Tier 1,Tier2,9.0,162.0,30


In [ ]:
trans1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1169600 entries, 0 to 2264599
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   doj                 1169600 non-null  object 
 1   doi                 1169600 non-null  object 
 2   srcid               1169600 non-null  int64  
 3   destid              1169600 non-null  int64  
 4   srcid_region        1169600 non-null  object 
 5   destid_region       1169600 non-null  object 
 6   srcid_tier          1169600 non-null  object 
 7   destid_tier         1169600 non-null  object 
 8   cumsum_seatcount    1169600 non-null  float64
 9   cumsum_searchcount  1169600 non-null  float64
 10  dbd                 1169600 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 107.1+ MB


# Data Preprocessing

In [ ]:
train1 = train.copy()

In [ ]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doj              67200 non-null  object 
 1   srcid            67200 non-null  int64  
 2   destid           67200 non-null  int64  
 3   final_seatcount  67200 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.1+ MB


In [ ]:
def data_preprocessing(train_df, transactions_df):
  """
  Performs comprehensive preprocessing and feature engineering on train data
  using transactions data.

  Args:
    train_df: The raw training DataFrame.
    transactions_df: The raw transactions DataFrame.

  Returns:
    The preprocessed and feature-engineered training DataFrame.
  """
  # Group transactions by route and day to get aggregates
  # Use 'cumsum_seatcount' from transactions_df for the sum
  transaction_aggregates = transactions_df.groupby(['doj', 'srcid', 'destid']).agg(
      num_transactions=('doj', 'size'), # Count transactions
      total_transaction_seats=('cumsum_seatcount', 'sum') # Sum cumsum_seatcount from the transaction data
  ).reset_index()

  # Merge these aggregates with the main data
  data_with_features = pd.merge(
      train_df, # Use a copy of train_df to avoid modifying the original
      transaction_aggregates,
      on=['doj', 'srcid', 'destid'],
      how='left'
  )

  # Handle potential missing values after merge
  data_with_features['num_transactions'].fillna(0, inplace=True)
  data_with_features['total_transaction_seats'].fillna(0, inplace=True)

  # Create average seats per transaction (handle division by zero)
  # Ensure division is between float types to avoid integer division issues
  data_with_features['avg_seats_per_transaction'] = data_with_features['total_transaction_seats'].astype(float) / data_with_features['num_transactions']
  data_with_features['avg_seats_per_transaction'].replace([np.inf, -np.inf], np.nan, inplace=True)
  data_with_features['avg_seats_per_transaction'].fillna(0, inplace=True)

  # 3. Preprocessing steps (date feature extraction, type conversion, drop original date)
  data_with_features['doj'] = pd.to_datetime(data_with_features['doj'])
  data_with_features['doj_month'] = data_with_features['doj'].dt.month
  data_with_features['doj_day'] = data_with_features['doj'].dt.day
  data_with_features['day_of_week'] = data_with_features['doj'].dt.dayofweek
  data_with_features['is_weekend'] = data_with_features['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
  # Keep 'final_seatcount' as it's the target variable in the training data
  data_with_features.drop(['doj'] , axis = 1 , inplace = True)

  return data_with_features

In [ ]:
preprocessed_train = data_preprocessing(train1 , trans1)

In [ ]:
preprocessed_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   srcid                      67200 non-null  int64  
 1   destid                     67200 non-null  int64  
 2   final_seatcount            67200 non-null  float64
 3   num_transactions           67200 non-null  int64  
 4   total_transaction_seats    67200 non-null  float64
 5   avg_seats_per_transaction  67200 non-null  float64
 6   doj_month                  67200 non-null  int32  
 7   doj_day                    67200 non-null  int32  
 8   day_of_week                67200 non-null  int32  
 9   is_weekend                 67200 non-null  int64  
dtypes: float64(3), int32(3), int64(4)
memory usage: 4.4 MB


In [ ]:
preprocessed_train.head()

,srcid,destid,final_seatcount,num_transactions,total_transaction_seats,avg_seats_per_transaction,doj_month,doj_day,day_of_week,is_weekend
0,45,46,2838.0,16,182.0,11.375,3,1,2,0
1,46,45,2298.0,16,366.0,22.875,3,1,2,0
2,45,47,2720.0,16,242.0,15.125,3,1,2,0
3,47,45,2580.0,16,102.0,6.375,3,1,2,0
4,46,9,4185.0,16,360.0,22.500,3,1,2,0


# Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = preprocessed_train.drop(['final_seatcount'] , axis = 1)
y = preprocessed_train['final_seatcount']

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2 , random_state = 42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(53760, 9)
(13440, 9)
(53760,)
(13440,)


# Evaluation Metrics

In [ ]:
from sklearn.metrics import root_mean_squared_error , r2_score

In [ ]:
def model_eval(actual , predicted):
  r2 = r2_score(actual , predicted)
  rmse = root_mean_squared_error(actual , predicted)
  print('R2 Score : ' , r2)
  print('RMSE : ' , rmse)

# Machine Learning Models

# XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xbg = XGBRegressor()
xbg.fit(x_train , y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_hat_train_xgb = xbg.predict(x_train)
y_hat_test_xgb = xbg.predict(x_test)

In [ ]:
model_eval(y_train , y_hat_train_xgb)

R2 Score :  0.8725318823957842
RMSE :  426.0676209540898


In [ ]:
model_eval(y_test , y_hat_test_xgb)

R2 Score :  0.8400347116273819
RMSE :  479.93095037893346


# CatBoost

In [ ]:
! pip install catboost -q

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cat = CatBoostRegressor()
cat.fit(x_train , y_train)

Learning rate set to 0.076841
0:	learn: 1158.5633910	total: 21.4ms	remaining: 21.4s
1:	learn: 1127.5643655	total: 55.5ms	remaining: 27.7s
2:	learn: 1099.7910053	total: 113ms	remaining: 37.6s
3:	learn: 1072.6651823	total: 145ms	remaining: 36.2s
4:	learn: 1049.1934714	total: 167ms	remaining: 33.2s
5:	learn: 1026.4317510	total: 194ms	remaining: 32.2s
6:	learn: 1007.6169391	total: 217ms	remaining: 30.8s
7:	learn: 989.0894474	total: 244ms	remaining: 30.3s
8:	learn: 973.6733590	total: 277ms	remaining: 30.5s
9:	learn: 958.8676357	total: 318ms	remaining: 31.5s
10:	learn: 943.5310558	total: 356ms	remaining: 32s
11:	learn: 931.4849207	total: 386ms	remaining: 31.8s
12:	learn: 921.1855677	total: 417ms	remaining: 31.6s
13:	learn: 907.9925966	total: 447ms	remaining: 31.5s
14:	learn: 894.9642731	total: 467ms	remaining: 30.7s
15:	learn: 886.0099532	total: 501ms	remaining: 30.8s
16:	learn: 877.7294038	total: 538ms	remaining: 31.1s
17:	learn: 869.5964774	total: 577ms	remaining: 31.5s
18:	learn: 862.5833

In [ ]:
y_hat_train_cat = cat.predict(x_train)
y_hat_test_cat = cat.predict(x_test)

In [ ]:
model_eval(y_train , y_hat_train_cat)

R2 Score :  0.8572241807691585
RMSE :  450.9257906136432


In [ ]:
model_eval(y_test , y_hat_test_cat)

R2 Score :  0.8432913956188793
RMSE :  475.0204458677239


# Test dataset

In [ ]:
test = pd.read_csv('/content/test_8gqdJqH.csv')

In [ ]:
preprocessed_test = data_preprocessing(test , trans1)

In [ ]:
preprocessed_test.head()

,route_key,srcid,destid,num_transactions,total_transaction_seats,avg_seats_per_transaction,doj_month,doj_day,day_of_week,is_weekend
0,2025-02-11_46_45,46,45,16,206.0,12.875,2,11,1,0
1,2025-01-20_17_23,17,23,16,0.0,0.000,1,20,0,0
2,2025-01-08_02_14,2,14,16,0.0,0.000,1,8,2,0
3,2025-01-08_08_47,8,47,16,0.0,0.000,1,8,2,0
4,2025-01-08_09_46,9,46,16,312.0,19.500,1,8,2,0


In [ ]:
preprocessed_test.drop(['route_key'] , axis = 1 , inplace = True)

In [ ]:
preprocessed_test['final_seatcount'] = cat.predict(preprocessed_test)

In [ ]:
preprocessed_test.head()

,srcid,destid,num_transactions,total_transaction_seats,avg_seats_per_transaction,doj_month,doj_day,day_of_week,is_weekend,final_seatcount
0,46,45,16,206.0,12.875,2,11,1,0,3443.633821
1,17,23,16,0.0,0.000,1,20,0,0,1201.175734
2,2,14,16,0.0,0.000,1,8,2,0,1236.454025
3,8,47,16,0.0,0.000,1,8,2,0,987.129594
4,9,46,16,312.0,19.500,1,8,2,0,3728.253893


# Sumbmission File

In [ ]:
submission = pd.read_csv('/content/sample_submission_TQv3O0x.csv')

In [ ]:
submission.head()

,route_key,final_seatcount
0,2025-02-11_46_45,0
1,2025-01-20_17_23,0
2,2025-01-08_02_14,0
3,2025-01-08_08_47,0
4,2025-01-08_09_46,0


In [ ]:
submission['final_seatcount'] = preprocessed_test['final_seatcount'].astype('int')

In [ ]:
submission.to_csv('/content/submission1.csv' , index = False)